<a href="https://colab.research.google.com/github/Anna172/ML/blob/master/%D0%90%D0%94%D0%92%D0%91_%D0%94%D0%972.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import math
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [6]:
data_train = pd.read_csv("train.csv", delimiter=',',  parse_dates=['period_start_dt'])
data_train.tail()

,Unnamed: 0,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG
35339,35537,40370,1380,2019-12-30,NaN,0.0,0.0,1000.00,1000.0,0.0,1.0
35340,35538,40372,1380,2019-12-30,NaN,0.0,0.0,2000.00,2000.0,0.0,1.0
35341,35539,40373,1380,2019-12-30,NaN,0.0,0.0,3000.00,3000.0,0.0,1.0
35342,35540,46272,1380,2019-12-30,NaN,1.0,0.0,284.29,199.0,0.0,1.0
35343,35541,96212,1380,2019-12-30,NaN,1.0,0.0,141.43,99.0,0.0,1.0


In [8]:
print(data_train[data_train['period_start_dt'].dt.year == 2020].shape[0])
print(data_train[ (data_train['period_start_dt'].dt.year == 2019) & (data_train['period_start_dt'].dt.month == 12) ].shape[0])
data_train.shape

0
1200


(35344, 11)

In [20]:
data_test = pd.read_csv("test.csv", delimiter=',',  parse_dates=['period_start_dt'])
data_test.tail()

,id,product_rk,store_location_rk,period_start_dt,demand
1399,35543,40370,1380,2020-06-01,NaN
1400,35544,40372,1380,2020-06-01,NaN
1401,35545,40373,1380,2020-06-01,NaN
1402,35546,46272,1380,2020-06-01,NaN
1403,35547,96212,1380,2020-06-01,NaN


In [18]:
print(data_test[data_test['period_start_dt'].dt.year == 2020].shape[0])
print(data_test[ (data_test['period_start_dt'].dt.year == 2019) & (data_test['period_start_dt'].dt.month == 12) ].shape[0])
data_test.shape

204
720


(1404, 5)

В тренировочном сете нет данных по 2020 году и в задании сказано, что нужно спрогнозировать только на декабрь 2019, поэтому удалим из тестового набора  данных все записи 20 года. 

Согласно заданию, нужно спрогнозировать только декабрь 2019 года, а в тестовом наборе данных 480 наблюдений не декабрь 2019 года, их тоже можно удалить.

В тренировочном наборе данных есть столбец без описания, возможно, это id. Проверим это предположение.

In [21]:
data_test = data_test[ (data_test['period_start_dt'].dt.year == 2019) & (data_test['period_start_dt'].dt.month == 12) ]
data_test.shape

(720, 5)

In [22]:
data_train['id'] = data_train['Unnamed: 0']
df = (data_train[(data_train['period_start_dt'].dt.year == 2019) & (data_train['period_start_dt'].dt.month == 12)].merge(data_test[(data_test['period_start_dt'].dt.year == 2019) & (data_test['period_start_dt'].dt.month == 12)], how='outer', on=['id','product_rk','store_location_rk', 'period_start_dt'], 
              suffixes=['', '_new'], indicator=True))

only2 = df[df['_merge'] == 'both']
print (only2)

      Unnamed: 0  product_rk  store_location_rk  ...     id  demand_new  _merge
12           920       40369                317  ...    920         NaN    both
13           921       40370                317  ...    921         NaN    both
14           922       40372                317  ...    922         NaN    both
15           923       40373                317  ...    923         NaN    both
16           924       46272                317  ...    924         NaN    both
...          ...         ...                ...  ...    ...         ...     ...
1195       35537       40370               1380  ...  35537         NaN    both
1196       35538       40372               1380  ...  35538         NaN    both
1197       35539       40373               1380  ...  35539         NaN    both
1198       35540       46272               1380  ...  35540         NaN    both
1199       35541       96212               1380  ...  35541         NaN    both

[720 rows x 14 columns]


Предположение подтвердилось, все 720 id по продукту, магазину и дате совпали с предпологаемым id по продукту, магазину и дате в тренировочном сете, поэтому можно переименовать неизвестную колонку в id.

In [23]:
data_train = data_train.drop('id', axis=1)
data_train.rename(columns={'Unnamed: 0': 'id'})
print(data_train.shape)
data_train.tail()

(35344, 11)


,Unnamed: 0,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG
35339,35537,40370,1380,2019-12-30,NaN,0.0,0.0,1000.00,1000.0,0.0,1.0
35340,35538,40372,1380,2019-12-30,NaN,0.0,0.0,2000.00,2000.0,0.0,1.0
35341,35539,40373,1380,2019-12-30,NaN,0.0,0.0,3000.00,3000.0,0.0,1.0
35342,35540,46272,1380,2019-12-30,NaN,1.0,0.0,284.29,199.0,0.0,1.0
35343,35541,96212,1380,2019-12-30,NaN,1.0,0.0,141.43,99.0,0.0,1.0


In [24]:
data_train.rename(columns={'Unnamed: 0': 'id'}) 
print(data_train.shape)
data_train.tail()

(35344, 11)


,Unnamed: 0,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG
35339,35537,40370,1380,2019-12-30,NaN,0.0,0.0,1000.00,1000.0,0.0,1.0
35340,35538,40372,1380,2019-12-30,NaN,0.0,0.0,2000.00,2000.0,0.0,1.0
35341,35539,40373,1380,2019-12-30,NaN,0.0,0.0,3000.00,3000.0,0.0,1.0
35342,35540,46272,1380,2019-12-30,NaN,1.0,0.0,284.29,199.0,0.0,1.0
35343,35541,96212,1380,2019-12-30,NaN,1.0,0.0,141.43,99.0,0.0,1.0
